In [1]:
import kfp
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google.cloud import aiplatform as aip
from google_cloud_pipeline_components import aiplatform as gcc_aip


In [2]:
aip.init(project="demogct", staging_bucket="gs://demogct/vipipelines/")

In [3]:
PROJECT_ID="demogct"
@kfp.dsl.pipeline(name="automl-image-training-v3")
def pipeline(project: str = PROJECT_ID):
    ds_op = gcc_aip.ImageDatasetCreateOp(
        project=project,
        display_name="flowers",
        gcs_source="gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv",
        import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
    )

    training_job_run_op = gcc_aip.AutoMLImageTrainingJobRunOp(
        project=project,
        display_name="train-automl-flowers",
        prediction_type="classification",
        model_type="CLOUD",
        base_model=None,
        dataset=ds_op.outputs["dataset"],
        model_display_name="train-automl-flowers",
        training_fraction_split=0.7,
        validation_fraction_split=0.2,
        test_fraction_split=0.1,
        budget_milli_node_hours=8000,
    )
    # 0.1.7 is needed currently to address a bug in the latest container image
    gcc_aip.ModelDeployOp.component_spec.implementation.container.image = ("gcr.io/ml-pipeline/google-cloud-pipeline-components:0.1.7")
 
    
    endpoint_op = gcc_aip.ModelDeployOp(  
         model=training_job_run_op.outputs["model"],
         project="demogct"
    )
  

In [46]:
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="image classification_pipeline.json".replace(" ", "_"),
)

In [51]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DISPLAY_NAME = "flowers_" + TIMESTAMP
PIPELINE_ROOT="gs://demogct/vipipelines/"

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="image classification_pipeline.json".replace(" ", "_"),
    pipeline_root=PIPELINE_ROOT,
)


In [ ]:
job.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/313681173937/locations/us-central1/pipelineJobs/automl-image-training-v2-20211030074450
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/313681173937/locations/us-central1/pipelineJobs/automl-image-training-v2-20211030074450')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/automl-image-training-v2-20211030074450?project=313681173937
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/313681173937/locations/us-central1/pipelineJobs/automl-image-training-v2-20211030074450 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/313681173937/loca